In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
import src.gen_source as gen_source
import src.two_tone_lib as tt
import src.dab_tuning_lib as dt

import src.tcp_async as tcp_async
import src.tcp_sync as tcp_sync

#from live_analyse_py import live_analyse_py

In [ ]:
try:
    __IPYTHON__
    reload(tcp_async)
    reload(tcp_sync)
    reload(gen_source)
    reload(tt)
    reload(td)
except:
    pass

In [ ]:
sync = tcp_sync.UhdSyncMsg(packet_size=4*8192,
                           packet_type="".join(["f"]*8192))
async = tcp_async.UhdAsyncMsg()

In [ ]:
top = live_analyse_py()

In [ ]:
top.start()

In [ ]:
sync.has_msg()
async.has_msg()

In [ ]:
top.set_txgain(85)
top.set_rxgain(15)

In [ ]:
tt.gen_two_tone(debug = True)

In [ ]:
a = np.fromfile("./input.dat")

In [ ]:
a.max()

In [ ]:
def measure(param, debug = False):
    #x2, x3, x4, x5, x6, x7, x8 = param
    x1, x2, x3, x4, x5, x6, x7, x8 = param + [1]
    
    repeat = True
    while repeat:
        #tt.gen_two_tone(debug = True, predist=tt.predist_poly, par=(x2, x3, x4))
        
        #top.dpd_memless_poly_0.set_a1(0.8)
        #top.dpd_memless_poly_0.set_a2(x2)
        #top.dpd_memless_poly_0.set_a3(x3)
        #top.dpd_memless_poly_0.set_a4(x4)
        #top.dpd_memless_poly_0.set_a5(x5)
        #top.dpd_memless_poly_0.set_a6(x6)
        #top.dpd_memless_poly_0.set_a7(x7)
        #top.dpd_memless_poly_0.set_a8(x8)
        
        top.dpd_lut_0.set_a1(x1)
        top.dpd_lut_0.set_a2(x2)
        top.dpd_lut_0.set_a3(x3)
        top.dpd_lut_0.set_a4(x4)
        top.dpd_lut_0.set_a5(x5)
        top.dpd_lut_0.set_a6(x6)
        top.dpd_lut_0.set_a7(x7)
        top.dpd_lut_0.set_a8(1)
        
        sync.has_msg()
        np.array(sync.get_msgs(3))
        msgs = np.array(sync.get_msgs(5))
        msgs = [np.fft.fftshift(msg) for msg in msgs]
        
        if async.has_msg():
            print ("repeat due to async message")
            continue
            
        a = np.array(msgs)
        mean_msg = a.mean(axis = 0)
        suffix = "x_2_%.3f_x_3_%.3f_x_4_%.3fx_5_%.3fx_6_%.3fx_7_%.3fx_8_%.3f" % \
                (x2, x3, x4, x5, x6, x7, x8)
        sig_to_noise = tt.analyse_power_spec(mean_msg, debug=debug, debug_path="/tmp/out", suffix=suffix)
        #sig_to_noise = dt.calc_signal_sholder_ratio(mean_msg, sampling_rate=8000000, debug=debug, debug_path="/tmp/out", suffix=suffix)
        print(sig_to_noise, x2, x3, x4, x5, x6, x7, x8)
        repeat = False
        
        return sig_to_noise

In [ ]:
def simple_opt(pars, i, d, func):
    par = pars[i]
    test_pars = []
    for x in [-3, -2, -1, 0, 1, 2, 3]:
        debug = x == 0
        new_par = list(pars)
        new_par[i] = par + x * d 
        test_pars.append([new_par, debug])
    res = [func(par_new, debug) for par_new, debug in test_pars]
    sel = np.argmax(res)
    best_par = zip(*test_pars)[0][sel]
    return best_par

#pars = [1,1,1]
#i_rand = np.random.randint(0, len(pars))
#pars = simple_opt(pars, i_rand, 0.01, lambda x:np.sum(x))
#pars

In [ ]:
#top.blocks_file_source_0.open("../dab_normalized_c64.dat", True)

In [ ]:
reload(dt)
top.set_txgain(84)
top.set_rxgain(5)

pars = np.array(range(1,8))/8.

for i in range(10000):
    i_rand = np.random.randint(0, len(pars))
    pars = simple_opt(pars, i_rand, 0.05, measure)

In [ ]:
sync.stop()
async.stop()
top.stop()
top.wait()